## Assignment : 14

<pre>
1. You can work with preprocessed_data.csv for the assignment. You can get the data from - <a href='https://drive.google.com/drive/u/0/folders/1CJnItndeSSJu7aragQoXWZS9-0apN6pp'>Data folder </a>
2. Load the data in your notebook.
3. After step 2 you have to train 3 types of models as discussed below. 
4. For all the model use <a href='https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics'>'auc'</a> as a metric. check <a  href='https://stackoverflow.com/a/46844409'>this</a> and <a  href='https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/80807'>this</a> for using auc as a metric 
5. You are free to choose any number of layers/hiddden units but you have to use same type of architectures shown below. 
6. You can use any one of the optimizers and choice of Learning rate and momentum.
7. For all the model's use <a href='https://www.youtube.com/watch?v=2U6Jl7oqRkM'>TensorBoard</a> and plot the Metric value and Loss with epoch. While submitting, take a screenshot of plots and include those images in a separate pad and write your observations about them.
8. Make sure that you are using GPU to train the given models.
</pre>

#Imports

In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import pickle
import tensorflow
from tensorflow.keras.layers import Input,Dense,LSTM,Flatten
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from numpy import zeros
from tensorflow.keras.layers import Dense,concatenate,Activation,Dropout,Input
from tensorflow.keras.models import Model

#Data

In [ ]:
#you can use gdown modules to import dataset for the assignment
#for importing any file from drive to Colab you can write the syntax as !gdown --id file_id
#you can run the below cell to import the required preprocessed data.csv file and glove vector

In [2]:
#you can use gdown modules to import dataset for the assignment
#for importing any file from drive to Colab you can write the syntax as !gdown --id file_id
#you can run the below cell to import the required preprocessed data.csv file and glove vector

!gdown --id 1GpATd_pM4mcnWWIs28-s1lgqdAg2Wdv-
!gdown --id 1pGd5tLwA30M7wkbJKdXHaae9tYVDICJ_

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1GpATd_pM4mcnWWIs28-s1lgqdAg2Wdv-
To: /content/preprocessed_data.csv
100% 124M/124M [00:00<00:00, 140MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1pGd5tLwA30M7wkbJKdXHaae9tYVDICJ_
To: /content/glove_vectors
100% 128M/128M [00:01<00:00, 110MB/s] 


##Preprocessing

## <font color='red'> Model-1 </font>
Build and Train deep neural network as shown below

<img src='https://i.imgur.com/w395Yk9.png'>
ref: https://i.imgur.com/w395Yk9.png

- __Input_seq_total_text_data__ --- You have to give Total text data columns. After this use the Embedding layer to get word vectors. Use given predefined glove word vectors, don't train any word vectors. After this use LSTM and get the LSTM output and Flatten that output. 
- __Input_school_state__ --- Give 'school_state' column as input to embedding layer and Train the Keras Embedding layer. 
- __Project_grade_category__  --- Give 'project_grade_category' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_categories__ --- Give 'input_clean_categories' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_subcategories__ --- Give 'input_clean_subcategories' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_subcategories__ --- Give 'input_teacher_prefix' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_remaining_teacher_number_of_previously_posted_projects._resource_summary_contains_numerical_digits._price._quantity__ ---concatenate remaining columns and add a Dense layer after that. 



Below is an example of embedding layer for a categorical columns. In below code all are dummy values, we gave only for referance. 

In [ ]:
# https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work
input_layer = Input(shape=(n,))
embedding = Embedding(no_1, no_2, input_length=n)(input_layer)
flatten = Flatten()(embedding)

### 1. Go through this blog, if you have any doubt on using predefined Embedding values in Embedding layer - https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
### 2. Please go through this link https://keras.io/getting-started/functional-api-guide/ and check the 'Multi-input and multi-output models' then you will get to know how to give multiple inputs. 

# <font color='red'> Model-1 </font>

##Data Splitt

In [4]:
#https://www.youtube.com/watch?v=Fuw0wv3X-0o&list=PLeo1K3hjS3uu7CxAacxVndI4bE_o3BDtO&index=40&ab_channel=codebasics

In [182]:
#read the csv file
df = pd.read_csv('preprocessed_data.csv')
print(df.shape)

(109248, 9)


In [6]:
df.columns

Index(['school_state', 'teacher_prefix', 'project_grade_category',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'clean_categories', 'clean_subcategories', 'essay', 'price'],
      dtype='object')

- Text Features
 - essay

- Categorical Features

  - school_state
  - teacher_prefix
  - project_grade_category
  - clean_categories
  - clean_sub_categories

- Numerical Features:
  - teacher_number_of_previously_posted_projects
  - Price

In [183]:
y = df['project_is_approved']
X = df.drop(columns=['project_is_approved'])
print('X shape:', X.shape,'y shape:',y.shape )

X shape: (109248, 8) y shape: (109248,)


In [184]:
# perform stratified train test split on the dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size= 0.2)
X_train,X_cv,y_train,y_cv = train_test_split(X_train,y_train,stratify=y_train,test_size= 0.2)

print('Train:',X_train.shape,y_train.shape)
print('CV:',X_cv.shape,y_cv.shape)
print('Test:',X_test.shape,y_test.shape)

Train: (69918, 8) (69918,)
CV: (17480, 8) (17480,)
Test: (21850, 8) (21850,)


## 1.1 Text Vectorization

In [185]:
#since the data is already preprocessed, we can directly move to vectorization part
#first we will vectorize the text data
#for vectorization of text data in deep learning we use tokenizer, you can go through below references
# https://www.kdnuggets.com/2020/03/tensorflow-keras-tokenization-text-data-prep.html
#https://stackoverflow.com/questions/51956000/what-does-keras-tokenizer-method-exactly-do
# after text vectorization you should get train_padded_docs and test_padded_docs

In [186]:
#Notes
#train_data = X_train['essay']
#test_data = X_test['essay']
#print(train_data.shape,test_data.shape)

#oov_token = '<UNK>'
#pad_type = 'post'
#trunc_type = 'post'

#tokenizer = Tokenizer( oov_token=oov_token)

#Converts the words to numbers ie each number is converted to number
#tokenizer.fit_on_texts(train_data)
#print('Vocab _length ',len(tokenizer.word_index))

#Transform the sent to list of number of sent with help of above
#train_sequences = tokenizer.texts_to_sequences(train_data)
#print('train_seq len',len(train_sequences))

#As the sent are of diff length we need to pad them with max
#maxlen = max([len(x) for x in train_sequences])
#print('max len',maxlen)

#Converts to matrix
#train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
#print('train_padded shape ',train_padded.shape)

#Glove Embedding:
#   docs = ['the is best work','keep the  work ']
# tokenizer = Tokenizer( oov_token=oov_token)
#   tokenizer.fit_on_texts(docs)   # fIts the docs
# train = tokenizer.texts_to_sequences(docs) # Converts the doc to seq [[2, 4, 5, 3], [6, 2, 3]]

#   maxlen = max([len(x) for x in train])   get max length for padd in thsi 4
#   tokenizer.word_index  
#    gives dic {'<UNK>': 1, 'best': 5, 'is': 4, 'keep': 6, 'the': 2, 'work': 3}

#with open('/content/glove_vectors', 'rb') as f:
    #glove = pickle.load(f)   # opens glove as dict of word and vectors of 300
#  vocab_size = len(tokenizer.word_index)+1  +1 as while indexing embedding mat last digit is not included
#  embedding_matrix = np.zeros((vocab_size, 300))   shape of 7,300
#  for word,i in tokenizer.word_index.items():
#   if i< vocab_size :
#   emb_vec=  glove.get(word)
#    if emb_vec is not None:
#      embedding_matrix[i] = emb_vec   # just gives the mat with word vectors
# embedding_matrix[6][:10]   glove.get('keep')[:10]
# Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
# 100 is vectro size, input_length is max_lenght used for padding

In [187]:
#Essay 

oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post' 

tokenizer = Tokenizer( oov_token=oov_token)
tokenizer.fit_on_texts(X_train['essay'].tolist())

train_sequences = tokenizer.texts_to_sequences(X_train['essay'])
cv_sequences = tokenizer.texts_to_sequences(X_cv['essay'])
test_sequences = tokenizer.texts_to_sequences(X_test['essay'])

maxlen = max([len(x) for x in train_sequences])

train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
cv_padded = pad_sequences(cv_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

print('Train padded',train_padded.shape)
print('cv padded',cv_padded.shape)
print('Test padded',test_padded.shape)
print('max_len',maxlen)


Train padded (69918, 339)
cv padded (17480, 339)
Test padded (21850, 339)
max_len 339


In [188]:
#after getting the padded_docs you have to use predefined glove vectors to get 300 dim representation for each word
# we will be storing this data in form of an embedding matrix and will use it while defining our model
# Please go through following blog's 'Example of Using Pre-Trained GloVe Embedding' section to understand 
#how to create embedding matrix
# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
#Now we have to convert the 640 298 train to glove rep of 640 298 300 as glove vectros are of 300

with open('/content/glove_vectors', 'rb') as f:
    glove = pickle.load(f)

vocab_size = len(tokenizer.word_index)+1
print(vocab_size)

embedding_matrix = np.zeros((vocab_size, 300))

for word,i in tokenizer.word_index.items():
  if i< vocab_size :
    emb_vec=  glove.get(word)
    if emb_vec is not None:
      embedding_matrix[i] = emb_vec

print(embedding_matrix.shape)

47267
(47267, 300)


In [189]:
maxlen

339

In [190]:
Input_essay = Input(shape = maxlen,name='Input_essay')
essay_emb = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen, trainable=False)(Input_essay)
essay_lstm = LSTM(units= 50,return_sequences=True)(essay_emb) 

essay_layer = Flatten()(essay_lstm)

## 1.2 Categorical feature Vectorization

In [191]:
# for model 1 and model 2, we have to assign a unique number to each feature in a particular categorical column.
# you can either use tokenizer,label encoder or ordinal encoder to perform the task
# label encoder gives an error for 'unseen values' (values present in test but not in train)
# handle unseen values with label encoder - https://stackoverflow.com/a/56876351
# ordinal encoder also gives error with unseen values but you can use modify handle_unknown parameter
# documentation of ordianl encoder https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html
# after categorical feature vectorization you will have column_train_data and column_test_data.


In [192]:
def token(col,X_train,X_cv,X_test):
  tokenizer = Tokenizer(oov_token='<UNK>')
  tokenizer.fit_on_texts(X_train[col].to_list()) 
  distinct_words = len(tokenizer.word_index) 
  
  t_train = tokenizer.texts_to_sequences(X_train[col])
  t_cv = tokenizer.texts_to_sequences(X_cv[col])
  t_test = tokenizer.texts_to_sequences(X_test[col])

  maxlen = max([len(x) for x in t_train])

  train_pad = pad_sequences(t_train, maxlen=maxlen)
  cv_pad = pad_sequences(t_cv, maxlen=maxlen)
  test_pad = pad_sequences(t_test, maxlen=maxlen)

  return train_pad,cv_pad,test_pad,maxlen,distinct_words


In [193]:
train_pad_ss,cv_pad_ss,test_pad_ss,max_ss,ss_words = token('school_state',X_train,X_cv,X_test)
train_pad_tp,cv_pad_tp,test_pad_tp,max_tp,tp_words = token('teacher_prefix',X_train,X_cv,X_test)
train_pad_pgc,cv_pad_pgc,test_pad_pgc,max_pgc,pgc_words = token('project_grade_category',X_train,X_cv,X_test)
train_pad_cc,cv_pad_cc,test_pad_cc,max_cc,cc_words = token('clean_categories',X_train,X_cv,X_test)
train_pad_cs,cv_pad_cs,test_pad_cs,max_cs,cs_words = token('clean_subcategories',X_train,X_cv,X_test)


In [194]:
ss_words


52

In [202]:
#school_state
Input_ss = Input(shape = max_ss,name='Input_ss')
ss_emd = Embedding(input_dim=ss_words,output_dim = 2,input_length= max_ss)(Input_ss)
ss_lstm = LSTM(units= 50,return_sequences=True)(ss_emd) 
ss_layer = Flatten()(ss_lstm)

#teacher_prefix
Input_tp = Input(shape = max_tp,name='Input_tp')
tp_emd = Embedding(input_dim=tp_words,output_dim=2,input_length=max_tp)(Input_tp)
tp_lstm = LSTM(units= 50,return_sequences=True)(tp_emd) 
tp_layer = Flatten()(tp_lstm)

#project_grade_category
Input_pgc = Input(shape = max_pgc,name='Input_pgc')
pgc_emd = Embedding(input_dim=pgc_words, output_dim=5,input_length=max_pgc)(Input_pgc)
pgc_lstm = LSTM(units= 50,return_sequences=True)(pgc_emd) 
pgc_layer = Flatten()(pgc_lstm)

#clean_categories
Input_cc = Input(shape = max_cc,name='Input_cc')
cc_emd = Embedding(input_dim=cc_words,output_dim=5,input_length=max_cc)(Input_cc)
cc_lstm = LSTM(units= 50,return_sequences=True)(cc_emd) 
cc_layer = Flatten()(cc_lstm)

#clean_sub_categories
Input_cs = Input(shape = max_cs,name='Input_cs')
cs_emd = Embedding(input_dim=cs_words,output_dim=5,input_length=max_cs)(Input_cs)
cs_lstm = LSTM(units= 50,return_sequences=True)(cs_emd) 
cs_layer = Flatten()(cs_lstm)


## 1.3 Numerical feature Vectorization

In [196]:
# you have to standardise the numerical columns
# stack both the numerical features
#after numerical feature vectorization you will have numerical_data_train and numerical_data_test

In [197]:
def num_encode(col,X_train,X_cv,X_testt):
  scaler = StandardScaler()

  train = scaler.fit_transform(X_train[col].values.reshape(-1,1))
  cv = scaler.transform(X_cv[col].values.reshape(-1,1))
  test = scaler.transform(X_test[col].values.reshape(-1,1))
  return train, cv,test

X_train_price,X_cv_price ,X_test_price = num_encode('price',X_train,X_cv,X_test)
X_train_tnp,X_cv_tnp ,X_test_tnp = num_encode('teacher_number_of_previously_posted_projects',X_train,X_cv,X_test)

train_num = np.concatenate((X_train_price,X_train_tnp),axis=1)
cv_num = np.concatenate((X_cv_price,X_cv_tnp),axis=1)
test_num = np.concatenate((X_test_price,X_test_tnp),axis=1)

In [198]:
num_col = Input(shape=(2,),name='num_col')
dense_num = Dense(8,activation='relu')(num_col)


## 1.4 Defining the model

<img src='https://i.imgur.com/w395Yk9.png'>

In [ ]:
# as of now we have vectorized all our features now we will define our model.
# as it is clear from above image that the given model has multiple input layers and hence we have to use functional API
# Please go through - https://keras.io/guides/functional_api/
# it is a good programming practise to define your complete model i.e all inputs , intermediate and output layers at one place.
# while defining your model make sure that you use variable names while defining any length,dimension or size.
#for ex.- you should write the code as 'input_text = Input(shape=(pad_length,))' and not as 'input_text = Input(shape=(300,))'
# the embedding layer for text data should be non trainable
# the embedding layer for categorical data should be trainable
# https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work
# https://towardsdatascience.com/deep-embeddings-for-categorical-variables-cat2vec-b05c8ab63ac0
#print model.summary() after you have defined the model
#plot the model using utils.plot_model module and make sure that it is similar to the above image

In [204]:
#concat laerys
concat = concatenate([essay_layer,ss_layer,tp_layer,pgc_layer,cc_layer,cs_layer,dense_num])

dense1 = Dense(128,activation="relu")(concat)
drop1 =Dropout(0.5)(dense1)
dense2 = Dense(256,activation="relu")(drop1)
drop2 =Dropout(0.5)(dense2)
dense3 = Dense(128,activation="relu")(drop2)

output = Dense(2, activation='softmax', name='output')(dense3)

model = Model(inputs=[Input_essay,Input_ss,Input_tp,Input_pgc,Input_cc,Input_cs,num_col],outputs=output)


model.compile(loss='categorical_crossentropy',  metrics=['accuracy'])
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_essay (InputLayer)       [(None, 339)]        0           []                               
                                                                                                  
 Input_ss (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 Input_tp (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 Input_pgc (InputLayer)         [(None, 3)]          0           []                               
                                                                                            

In [200]:
from tensorflow.keras.utils import to_categorical

In [205]:
train_data = [train_padded,train_pad_ss,train_pad_tp,train_pad_pgc,train_pad_cc,train_pad_cs,train_num]
cv_data = [cv_padded,cv_pad_ss,cv_pad_tp,cv_pad_pgc,cv_pad_cc,cv_pad_cs,cv_num]
train_data = [test_padded,test_pad_ss,test_pad_tp,test_pad_pgc,test_pad_cc,test_pad_cs,test_num]


model_1 = model.fit([train_padded,train_pad_ss,train_pad_tp,train_pad_pgc,train_pad_cc,train_pad_cs,train_num],to_categorical(y_train),
                        epochs=2,verbose=1)

Epoch 1/2


InvalidArgumentError: ignored

## 1.5 Compiling and fititng your model

In [ ]:
#define custom auc as metric , do not use tf.keras.metrics
# https://stackoverflow.com/a/46844409 - custom AUC reference 1
# https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/80807  - custom AUC reference 2
# compile and fit your model

# <font color='red'> Model-2 </font>

Use the same model as above but for 'input_seq_total_text_data' give only some words in the sentance not all the words. Filter the words as below. 

<pre>
1. Fit TF-IDF vectorizer on the Train data <br>
2. Get the idf value for each word we have in the train data. Please go through <a  href='https://stackoverflow.com/questions/23792781/tf-idf-feature-weights-using-sklearn-feature-extraction-text-tfidfvectorizer'>this</a><br>

3. Do some analysis on the Idf values and based on those values choose the low and high threshold value. Because very 
frequent words and very very rare words don't give much information.
Hint - A preferable IDF range is 2-11 for model 2. <br>
4.Remove the low idf value and high idf value words from the train and test data. You can go through each of the
sentence of train and test data and include only those features(words) which are present in the defined IDF range.
5. Perform tokenization on the modified text data same as you have done for previous model.
6. Create embedding matrix for model 2 and then use the rest of the features similar to previous model.
7. Define the model, compile and fit the model.
</pre>

# <font color='red'> Model-3 </font>

<img src='https://i.imgur.com/fkQ8nGo.png'>
ref: https://i.imgur.com/fkQ8nGo.png

In [ ]:
#in this model you can use the text vectorized data from model1 
#for other than text data consider the following steps
# you have to perform one hot encoding of categorical features. You can use onehotencoder() or countvectorizer() for the same.
# Stack up standardised numerical features and all the one hot encoded categorical features
#the input to conv1d layer is 3d, you can convert your 2d data to 3d using np.newaxis
# Note - deep learning models won't work with sparse features, you have to convert them to dense features before fitting in the model.